In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = GermanDataset(device=device)

In [3]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_kde_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_kde_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_kde_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_kde_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 256
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 1.1

hyperparameter_test(dataset, SGD_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 310.08epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 329.94epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 218.82epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 315.43epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  ----------  ----------  ---------
         0.0001          0          0.695312         0.7875   0.0153846   0.00432692  0.00190278  0.0495495
         0.001           0          0.757812         0.76875  0.040404    0.0625      0.0595053   0.0595053
         0.01            0          0.7875           0.73125  0.00420168  0.0951923   0.0501643   0.177177
         0.1             0          0.782813         0.7375   0.0352304   0.0975962   0.0683273   0.124625


The learning rate is decided as 0.01.

In [5]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.01
SGD_hp['lambda_'] = 0


#### EI Hyperparameter Test

In [6]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0, 0.05, 0.1, 0.2, 0.3]
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 151.02epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 187.66epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training:   0%|          | 0/100 [00:00<?, ?epochs/s]/opt/homebrew/Caskroom/miniforge/base/envs/sampler_fairness/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/sampler_fairness/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/ozgurguldogan/Desktop/research/fairness/ei_fairness_v2/utils.py:68: RuntimeWarning: invalid value encountered in long_scalars
  ei_z = abs(n_eyz[(1,0,z)]/(n_eyz[(0,0,z)] + n_eyz[(1,0,z)]) - p10)
Training: 100%|██████████| 100/100 [00:00<00:00, 205.76epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 187.42epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ----------  ---------  ---------
           0.01       0             0.789062         0.73125  0.00420168  0.0951923   0.0501643  0.177177
           0.01       0.05          0.790625         0.73125  0.00420168  0.0951923   0.0501643  0.177177
           0.01       0.1           0.790625         0.73125  0.00420168  0.0951923   0.0501643  0.177177
           0.01       0.2           0.682813         0.7875   0           0.00336538  0          0.0255255
           0.01       0.3           0.682813         0.775    0           0.00913462  0          0.0285285


The lambda for EI is decided as 0.1.

In [7]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.01
EI_hp['lambda_'] = 0.1

#### DP Hyperparameter Selection   

In [8]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [0.01]
DP_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 182.27epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 150.15epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 207.98epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 203.05epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ----------  ---------
           0.01       0             0.7875           0.73125  0.00420168  0.0951923  0.0501643   0.177177
           0.01       0.2           0.790625         0.7375   0.0121212   0.0706731  0.0157412   0.177177
           0.01       0.6           0.771875         0.74375  0           0.0461538  0.00259471  0.121622
           0.01       0.8           0.770312         0.75     0.0301075   0.0370192  0.0105518   0.121622
           0.01       0.9           0.754687         0.74375  0.0125      0.0307692  0.00259471  0.0660661
           0.01       0.95          0.751563         0.7625   0.0869565   0.0100962  0.0124546   0.036036


The lambda for DP is decided as 0.9.

In [9]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.01
DP_hp['lambda_'] = 0.9

#### EO Hyperparameter Selection

In [10]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [0.01]
EO_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 208.66epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 203.53epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 197.59epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 205.96epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  ----------  ----------  ---------
           0.01       0             0.7875           0.73125  0.00420168  0.0951923   0.0501643   0.177177
           0.01       0.2           0.789062         0.73125  0.00420168  0.0951923   0.0501643   0.177177
           0.01       0.6           0.792188         0.74375  0.00735294  0.0644231   0.0157412   0.15015
           0.01       0.8           0.784375         0.74375  0           0.0461538   0.00259471  0.121622
           0.01       0.9           0.767188         0.74375  0.0824176   0.0557692   0.00242173  0.175676
           0.01       0.95          0.767188         0.74375  0.027972    0.00673077  0.00570835  0.0195195


The lambda for EO is decided as 0.9.

In [11]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.01
EO_hp['lambda_'] = 0.9

#### EODD Hyeprparameter Selection

In [12]:
EODD_hp_test = SGD_hp_test.copy()
EODD_hp_test['learning_rate'] = [0.01]
EODD_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EODD_hp_test['fairness'] = 'EODD'

hyperparameter_test(dataset, EODD_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 150.71epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 136.63epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 129.27epochs/s]


Training finished for all seeds.
training seed 1 started


Training:  15%|█▌        | 15/100 [00:00<00:00, 146.58epochs/s]/Users/ozgurguldogan/Desktop/research/fairness/ei_fairness_v2/utils.py:65: RuntimeWarning: invalid value encountered in long_scalars
  p10 = sum([n_eyz[(1,0,z)] for z in z_set]) / sum([n_eyz[(1,0,z)]+n_eyz[(0,0,z)] for z in z_set])
Training: 100%|██████████| 100/100 [00:00<00:00, 144.61epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp          eo        eodd
---------------  ---------  ----------------  --------------  ----------  ----------  ----------  ----------
           0.01       0             0.7875           0.73125  0.00420168  0.0951923   0.0501643   0.177177
           0.01       0.2           0.789062         0.74375  0.00833333  0.0615385   0.00259471  0.177177
           0.01       0.6           0.785937         0.74375  0.00735294  0.0644231   0.0157412   0.15015
           0.01       0.8           0.778125         0.73125  0.00735294  0.0644231   0.0501643   0.0660661
           0.01       0.9           0.771875         0.725    0.024024    0.0456731   0.0551808   0.0551808
           0.01       0.95          0.692187         0.775    0           0.00336538  0.00813008  0.00813008


The lambda for EODD is picked as 0.8.

In [13]:
EODD_hp = EODD_hp_test.copy()
EODD_hp['learning_rate'] = 0.01
EODD_hp['lambda_'] = 0.8

#### Model training

In [18]:
seeds = np.arange(1,6)

SGD, EI, DP, EO, EODD = experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds)

/var/folders/mt/mgpv1mld71dfr_kn_fq57str0000gn/T/ipykernel_6724/2364504375.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 100/100 [00:00<00:00, 284.46epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:00<00:00, 332.02epochs/s]


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:00<00:00, 157.71epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:00<00:00, 146.05epochs/s]

Training finished for all seeds.


In [19]:
result = []
models = ["SGD", "EI", "DP", "EO", "EODD"]
sol = [SGD, EI, DP, EO, EODD]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var    ei_mean     ei_var    dp_mean     dp_var    eo_mean      eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ---------  ---------  ---------  ---------  ---------  ----------  -----------  ----------
SGD                0.783      0.00812404  0.0381984  0.0103805  0.081679   0.0103819  0.0658341  0.0114319     0.0686672  0.0106232
EI                 0.767      0.0364143   0.0354006  0.0184084  0.0730123  0.0295603  0.0545405  0.0219796     0.0684276  0.0256439
DP                 0.793      0.00812404  0.11774    0.0465535  0.0368889  0.0074315  0.0326049  0.00811333    0.0381698  0.00922644
EO                 0.77       0.00447214  0.0740323  0.0427448  0.0486049  0.0152698  0.0444686  0.00861352    0.0521347  0.00649704
EODD               0.768      0.00812404  0.0761527  0.0370436  0.0464815  0.0200765  0.049491   0.015129      0.057933   0.011495
